In [1]:
import sys
sys.path.append('/home/lessa/smodels')
from smodels.experiment.databaseObj import Database
from smodels.tools.physicsUnits import GeV,fb,pb
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

In [14]:
print np.finfo(float).eps
print sys.float_info.epsilon

2.22044604925e-16
2.22044604925e-16


In [2]:
db = Database('/home/lessa/smodels-database')

In [4]:
exp = db.getExpResults(analysisIDs=['ATLAS-SUSY-2013-04'],dataTypes=['efficiencyMap'],txnames=['T5WWoff'],
                       useSuperseded=True,useNonValidated=True,datasetIDs=['GtGrid_SR_7ej80_0bjet'])[0]

In [5]:
tx = exp.getTxNames()[0]
print tx

T5WWoff


In [6]:
data = tx.txnameData._data

In [ ]:
# new_data = []
# for pt in data:
#     new_data.append(pt)
#     new_data[-1][0][0][0] = 0.96*new_data[-1][0][1][0]
# tx.txnameData._data =  new_data
# tx.txnameData._V = None
# tx.txnameData.computeV()

In [7]:
# xvals = [pt[0][0][0].asNumber(GeV) for pt in data]
# yvals = [pt[0][0][-1].asNumber(GeV) for pt in data]
# zvals = [pt[1] for pt in data]

In [ ]:
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# ax.scatter(xvals, yvals, zvals)
# ax.set_xlabel('M1')
# ax.set_ylabel('m')
# ax.set_zlabel('UL')

# plt.show()

In [12]:
m =[[571.157926242*GeV, 534.583889495*GeV, 498.009852749*GeV]]*2
porig = tx.txnameData.flattenMassArray(m)
pt = ( (np.matrix(porig)[0] - tx.txnameData.delta_x ) ).tolist()[0]
pt = np.dot(pt,tx.txnameData._V)  ## rotate
pt = np.array(pt[:tx.txnameData.dimensionality])
print 'Pt=',pt,'val=',tx.txnameData.getValueFor(m)
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# ptm = [pt[0],pt[1],tx.txnameData.getValueFor(m).asNumber(pb)]
# ax.scatter([ptm[0]], [ptm[1]], [ptm[2]],c='r',marker='o')
xpts = []
ypts = []
zpts = []
simplex = tx.txnameData.tri.find_simplex(pt)
if simplex == [-1]:
    print simplex
else:
    for ipt in tx.txnameData.tri.simplices[tx.txnameData.tri.find_simplex(pt)]:
        xpts.append(tx.txnameData.tri.points[ipt][0])
        ypts.append(tx.txnameData.tri.points[ipt][1])
        if tx.txnameData.dimensionality == 3:
            zpts.append(tx.txnameData.tri.points[ipt][2])
        else:
            zpts.append(tx.txnameData.xsec[ipt])
        print tx.txnameData.tri.points[ipt],tx.txnameData.xsec[ipt]

Pt= [-257.91076473  426.62764755   13.96399637] val= -7.53105444673e-16
954
[-228.1692754   388.37365908  -18.81009438] 1.84e-05
[-222.6727301   401.58398818   36.37391775] 0.0
[-269.24793409  443.52695656    2.88454963] 0.0
[-282.95611039  400.45830122   17.27818815] 0.0


In [15]:
print tx.txnameData.countNonZeros(pt),tx.txnameData.dimensionality

3 3


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(xpts, ypts, zpts,s=80,c='r',marker='o')
ax.scatter([pt[0]],[pt[1]],[pt[2]],s=80,c='b',marker='o')
ax.set_xlabel('x')
ax.set_ylabel('y')
if tx.txnameData.dimensionality == 3:
    ax.set_zlabel('UL')
else:
    ax.set_zlabel('z')

plt.show()

In [19]:
uvw = [pt[:3]]
fill_value=np.nan
tol = sys.float_info.epsilon * 1e06
simplex = tx.txnameData.tri.find_simplex(uvw, tol=tol)
vertices = np.take(tx.txnameData.tri.simplices, simplex, axis=0)
temp = np.take(tx.txnameData.tri.transform, simplex, axis=0)
d=temp.shape[2]
delta = uvw - temp[:, d]
bary = np.einsum('njk,nk->nj', temp[:, :d, :], delta)
tx.txnameData.vtx = vertices
tx.txnameData.wts = np.hstack((bary, 1 - bary.sum(axis=1, keepdims=True)))
v=tx.txnameData.xsec
if type(tx.txnameData.xsec[0]) == float:
    values = np.array ( [ float(x) for x in tx.txnameData.xsec ] )
else:
    values = np.array ( [ x.asNumber() for x in tx.txnameData.xsec ] )
print 'w=',tx.txnameData.wts
ret = np.einsum('nj,nj->n', np.take(values, tx.txnameData.vtx), tx.txnameData.wts)
print 'r=',ret
print 'vals=',np.take(values, tx.txnameData.vtx)
with np.errstate(invalid='ignore'):
    ret[np.any(tx.txnameData.wts < -1e-10, axis=1)] = fill_value
print ret

w= [[  2.78948156e-01   6.00328425e-01   1.20723419e-01  -2.05766515e-11]]
r= [ -7.53105445e-16]
vals= [[  0.00000000e+00   0.00000000e+00   0.00000000e+00   3.66000000e-05]]
[ -7.53105445e-16]


In [ ]:
fig.clear()
plt.triplot(xpts,ypts)
plt.plot([ptm[0]], [ptm[1]],'ro',xpts,ypts,'bo')
plt.show()

In [ ]:
tri = tx.txnameData.tri
simplex = tri.find_simplex([pt])
vertices = np.take(tri.simplices, simplex, axis=0)
temp = np.take(tri.transform, simplex, axis=0)
d=temp.shape[2]
delta = [pt] - temp[:, d]
bary = np.einsum('njk,nk->nj', temp[:, :d, :], delta)
vtx = vertices
wts = np.hstack((bary, 1 - bary.sum(axis=1, keepdims=True)))
v=tx.txnameData.xsec
values = np.array ( [ x.asNumber() for x in tx.txnameData.xsec ] )
ret = np.einsum('nj,nj->n', np.take(values, vtx), wts)
with np.errstate(invalid='ignore'):
    ret[np.any(wts < -1e-10, axis=1)] = np.nan
print ret[0]

In [ ]:
vertices

In [ ]:
pt

In [ ]:
m = [[347.406576078*GeV,166.767198233*GeV],[347.406576078*GeV,166.767198233*GeV]]
porig = tx.txnameData.flattenMassArray(m)
ptA = ( (np.matrix(porig)[0] - tx.txnameData.delta_x ) ).tolist()[0]
ptA = np.dot(ptA,tx.txnameData._V)  ## rotate
print ptA,tx.txnameData.countNonZeros(ptA)

In [ ]:
print tx.txnameData.getValueFor(m)